In [1]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import numpy as np
import hummingbird

#### Parameters for model

In [2]:
max_depth=7
num_classes=2
n_estimators=500
n_features=200
data_sz=200000
print("For #classes:{}, depth:{}, n_estimators/trees:{}".format(num_classes, max_depth, n_estimators))

For #classes:2, depth:7, n_estimators/trees:500


#### Create and fit the model

In [3]:
skl_model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
X = np.random.rand(data_sz, n_features).astype('float32')
y = np.random.randint(num_classes, size=data_sz)

# Larger than normal test_size for benchmarking
X_train, X_test, y_train, _ = train_test_split(X, y, test_size=0.80)

In [4]:
skl_model.fit(X_train, y_train) # This will take a bit


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=7, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

#### Time scikit-learn

In [5]:
%%timeit 
skl_model.predict(X_test)

8.2 s ± 22 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Convert SKL model to PyTorch

In [6]:
pytorch_model = skl_model.to('pytorch')

#### Time PyTorch - CPU

In [7]:
%%timeit 
pytorch_model.predict(X_test)

3.98 s ± 28.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


#### Switch PyTorch from CPU to GPU

In [8]:
pytorch_model.to('cuda')

PyTorchBackendModel(
  (operator_map): ModuleDict(
    (SklearnRandomForestClassifier): PerfectTreeTraversalDecisionTreeImpl(
      (nodes): ParameterList(
          (0): Parameter containing: [torch.cuda.LongTensor of size 1000 (GPU 0)]
          (1): Parameter containing: [torch.cuda.LongTensor of size 2000 (GPU 0)]
          (2): Parameter containing: [torch.cuda.LongTensor of size 4000 (GPU 0)]
          (3): Parameter containing: [torch.cuda.LongTensor of size 8000 (GPU 0)]
          (4): Parameter containing: [torch.cuda.LongTensor of size 16000 (GPU 0)]
          (5): Parameter containing: [torch.cuda.LongTensor of size 32000 (GPU 0)]
      )
      (biases): ParameterList(
          (0): Parameter containing: [torch.cuda.FloatTensor of size 1000 (GPU 0)]
          (1): Parameter containing: [torch.cuda.FloatTensor of size 2000 (GPU 0)]
          (2): Parameter containing: [torch.cuda.FloatTensor of size 4000 (GPU 0)]
          (3): Parameter containing: [torch.cuda.FloatTensor o

#### Time PyTorch - GPU

In [9]:
%%timeit 
pytorch_model.predict(X_test)

143 ms ± 181 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
